In [1]:
#%%

# =============================================================================
# =============================================================================
# # 문제 01 유형(DataSet_01.csv 이용)
#
# 구분자 : comma(“,”), 4,572 Rows, 5 Columns, UTF-8 인코딩
# 
# 글로벌 전자제품 제조회사에서 효과적인 마케팅 방법을 찾기
# 위해서 채널별 마케팅 예산과 매출금액과의 관계를 분석하고자
# 한다.
# 컬 럼 / 정 의  /   Type
# TV   /     TV 마케팅 예산 (억원)  /   Double
# Radio / 라디오 마케팅 예산 (억원)  /   Double
# Social_Media / 소셜미디어 마케팅 예산 (억원)  / Double
# Influencer / 인플루언서 마케팅
# (인플루언서의 영향력 크기에 따라 Mega / Macro / Micro / 
# Nano) / String

# SALES / 매출액 / Double
# =============================================================================
# =============================================================================


In [2]:
import pandas as pd
# pandas, scipy, numpy, sklearn, statsmadels

In [3]:
dat = pd.read_csv('Dataset/Dataset_01.csv')

In [4]:
dat

,TV,Radio,Social_Media,Influencer,Sales
0,16.0,6.566231,2.907983,Mega,54.732757
1,13.0,9.237765,2.409567,Mega,46.677897
2,41.0,15.886446,2.913410,Mega,150.177829
3,83.0,30.020028,6.922304,Mega,298.246340
4,15.0,8.437408,1.405998,Micro,56.594181
...,...,...,...,...,...
4567,26.0,4.472360,0.717090,Micro,94.685866
4568,71.0,20.610685,6.545573,Nano,249.101915
4569,44.0,19.800072,5.096192,Micro,163.631457
4570,71.0,17.534640,1.940873,Macro,253.610411


In [6]:
#%%

# =============================================================================
# 1. 데이터 세트 내에 총 결측값의 개수는 몇 개인가? (답안 예시) 23
# =============================================================================

In [13]:
dat.isnull().sum() # 열별 결측치 수 확인

TV              10
Radio            4
Social_Media     6
Influencer       0
Sales            6
dtype: int64

In [19]:
dat.isnull().sum(axis=1) #행별 결측치 수 확인

0       0
1       0
2       0
3       0
4       0
       ..
4567    0
4568    0
4569    0
4570    0
4571    0
Length: 4572, dtype: int64

In [22]:
dat.isnull().any(axis=1).sum() # 행 중에 null값이 하나라도 들어간 경우 합

26

In [24]:
print('정답 :',dat.isnull().sum().sum())

정답 : 26


---

In [7]:
#%% 상관관계 문제 => corr()

# =============================================================================
# 2. TV, Radio, Social Media 등 세 가지 다른 마케팅 채널의 예산과 매출액과의 상관분석을
# 통하여 각 채널이 매출에 어느 정도 연관이 있는지 알아보고자 한다. 
# - 매출액과 가장 강한 상관관계를 가지고 있는 채널의 상관계수를 소수점 5번째
# 자리에서 반올림하여 소수점 넷째 자리까지 기술하시오. (답안 예시) 0.1234
# =============================================================================

In [8]:
# 음수가 있는 경우가 있기 때문에 절댓값을 취한다 abs()
Q2 = dat[['TV','Radio','Social_Media','Sales']].corr().abs()

In [9]:
Q2

,TV,Radio,Social_Media,Sales
TV,1.000000,0.869460,0.528168,0.999497
Radio,0.869460,1.000000,0.607452,0.869105
Social_Media,0.528168,0.607452,1.000000,0.528906
Sales,0.999497,0.869105,0.528906,1.000000


In [35]:
print('정답 : ',Q2['Sales'].sort_values(ascending=False)[1].round(5))

정답 :  0.9995


In [38]:
# 상위 높은 데이터를 가져오기 (nlargest)
Q2['Sales'].nlargest(2)[1].round(5)

0.9995

---

In [ ]:
#%% 회귀분석 문제

# =============================================================================
# 3. 매출액을 종속변수, TV, Radio, Social Media의 예산을 독립변수로 하여 회귀분석을
# 수행하였을 때, 세 개의 독립변수의 회귀계수를 큰 것에서부터 작은 것 순으로
# 기술하시오. 

# - 분석 시 결측치가 포함된 행은 제거한 후 진행하며, 회귀계수는 소수점 넷째 자리
# 이하는 버리고 소수점 셋째 자리까지 기술하시오. (답안 예시) 0.123
# from sklearn.linear_model import LinearRegression 사용하시오.
# =============================================================================

In [77]:
# fit_intercept => 절편

from sklearn.linear_model import LinearRegression
from statsmodels.api import OLS, add_constant
from statsmodels.formula.api import ols

# 결측치 제거
dat1 = dat.dropna()

lm = LinearRegression(fit_intercept=True)
lm.fit(dat1.drop(columns=['Influencer','Sales']),dat1.Sales)

# 회귀 계수
dir(lm)
lm.coef_
# array([ 3.56256963, -0.00397039,  0.00496402])

answer = sorted(lm.coef_, reverse=True)
print('정답 :',list(map(lambda x : x.round(4), answer)))


정답 : [3.5626, 0.005, -0.004]


In [59]:
# const column 추가
X = dat1.drop(columns=['Influencer','Sales'])
XX = add_constant(X)
y = dat1.Sales

ols1 = OLS(y, X).fit() # 상수항 미포함
ols1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  Sales   R-squared (uncentered):                   1.000
Model:                            OLS   Adj. R-squared (uncentered):              1.000
Method:                 Least Squares   F-statistic:                          7.951e+06
Date:                Mon, 23 Aug 2021   Prob (F-statistic):                        0.00
Time:                        10:47:40   Log-Likelihood:                         -11367.
No. Observations:                4546   AIC:                                  2.274e+04
Df Residuals:                    4543   BIC:                                  2.276e+04
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
TV               3.5609      0.003   1133.941      0.000       3.555       3.567
Radio           -0.0039      0.010     -0.400      0.689      -0.023       0.015
Social_Media    -0.0013      0.024     -0.054      0.957      -0.049       0.047
==============================================================================
Omnibus:                        0.061   Durbin-Watson:                   1.999
Prob(Omnibus):                  0.970   Jarque-Bera (JB):                0.038
Skew:                          -0.001   Prob(JB):                        0.981
Kurtosis:                       3.014   Cond. No.                         35.8
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [71]:
# 절편이 들어간 상태여야 한다.

ols2 = OLS(y, XX).fit() # 상수항 포함
ols2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Sales   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                 1.505e+06
Date:                Mon, 23 Aug 2021   Prob (F-statistic):               0.00
Time:                        11:17:26   Log-Likelihood:                -11366.
No. Observations:                4546   AIC:                         2.274e+04
Df Residuals:                    4542   BIC:                         2.277e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -0.1340      0.103     -1.303      0.193      -0.336       0.068
TV               3.5626      0.003   1051.118      0.000       3.556       3.569
Radio           -0.0040      0.010     -0.406      0.685      -0.023       0.015
Social_Media     0.0050      0.025      0.199      0.842      -0.044       0.054
==============================================================================
Omnibus:                        0.056   Durbin-Watson:                   1.998
Prob(Omnibus):                  0.972   Jarque-Bera (JB):                0.034
Skew:                          -0.001   Prob(JB):                        0.983
Kurtosis:                       3.013   Cond. No.                         149.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [76]:
ols3 = ols('Sales~TV+Radio+Social_Media', data=dat1).fit()
ols3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Sales   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                 1.505e+06
Date:                Mon, 23 Aug 2021   Prob (F-statistic):               0.00
Time:                        11:42:48   Log-Likelihood:                -11366.
No. Observations:                4546   AIC:                         2.274e+04
Df Residuals:                    4542   BIC:                         2.277e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -0.1340      0.103     -1.303      0.193      -0.336       0.068
TV               3.5626      0.003   1051.118      0.000       3.556       3.569
Radio           -0.0040      0.010     -0.406      0.685      -0.023       0.015
Social_Media     0.0050      0.025      0.199      0.842      -0.044       0.054
==============================================================================
Omnibus:                        0.056   Durbin-Watson:                   1.998
Prob(Omnibus):                  0.972   Jarque-Bera (JB):                0.034
Skew:                          -0.001   Prob(JB):                        0.983
Kurtosis:                       3.013   Cond. No.                         149.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [75]:
var_list = dat1.columns[:-1]

form = 'Sales~' + '+'.join(var_list)

ols4 = ols(form, data=dat1).fit()
ols4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Sales   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                 7.521e+05
Date:                Mon, 23 Aug 2021   Prob (F-statistic):               0.00
Time:                        11:42:45   Log-Likelihood:                -11366.
No. Observations:                4546   AIC:                         2.275e+04
Df Residuals:                    4539   BIC:                         2.279e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              -0.1033      0.130     -0.797      0.426      -0.358       0.151
Influencer[T.Mega]      0.0116      0.124      0.094      0.926      -0.232       0.255
Influencer[T.Micro]    -0.0569      0.124     -0.458      0.647      -0.300       0.187
Influencer[T.Nano]     -0.0739      0.125     -0.593      0.553      -0.318       0.170
TV                      3.5626      0.003   1050.472      0.000       3.556       3.569
Radio                  -0.0039      0.010     -0.399      0.690      -0.023       0.015
Social_Media            0.0045      0.025      0.179      0.858      -0.044       0.053
==============================================================================
Omnibus:                        0.067   Durbin-Watson:                   1.998
Prob(Omnibus):                  0.967   Jarque-Bera (JB):                0.043
Skew:                          -0.001   Prob(JB):                        0.979
Kurtosis:                       3.015   Cond. No.                         288.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""